In [1]:
file_path = '/Users/lfinkbeiner/Documents/GitHub/HERA/beam/'
beam_origin = 'HERA_4.9m_E-pattern_151MHz.txt'
beam_destination = 'ant1_s2'

In [2]:
%matplotlib notebook
import matplotlib.pyplot as plt
from RIMEz import beam_models
import numpy as np
import healpy as hp
from spin1_beam_model import cst_processing, jones_matrix_field

In [3]:
processor = cst_processing.CSTDataProcessor(
    [file_path + beam_origin,
    file_path + beam_origin,
    file_path + beam_origin],
    np.array([150e6, 151e6, 152e6]),
    1, 1e-4
)

In [4]:
processor.compute_spin1_harmonics()
processor.write_model_data(file_path, beam_destination)
spline_beam_func = beam_models.model_data_to_spline_beam_func(file_path + beam_destination + '.h5', np.array([150e6, 151e6, 152e6]))

//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:18: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.pos1_Elm_samples = h5f["pos1_Elm"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:19: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.neg1_Elm_samples = h5f["neg1_Elm"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:20: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.freq_nodes = h5f["frequencies"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:21: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.L_model = h5f["spatial_bandlimit"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:22: H5pyDeprec

delta_nu_in is 1000000.0


In [5]:
import healpy as hp
#nside = 16, 32, 64, 128...
nside = 128

# healpix does the interpolation for you?
theta, phi = hp.pix2ang(nside,np.arange(12 * nside * nside))
az = phi
alt = np.pi / 2 - theta
J = spline_beam_func(150e6, alt, az)

In [6]:
# Current labels
# 0, 0 : xy
# 1, 0 : xx
# 0, 1 : yy
# 1, 1 : yx

J_abs = np.abs(J)

In [7]:
import rotations

In [10]:
print('HERA latitude:', rotations.hera_lat)
print('HERA longitude:', rotations.hera_lon)
print('Current LST:', rotations.get_lst())

HERA latitude: -30.72138888888889
HERA longitude: 21.428333333333335
Current LST: 4.500999885012081


In [12]:
el, be = rotations.eq_to_gal(rotations.hera_lat, rotations.hera_lon)
print('With right ascension = HERA latitude and declination = HERA longitude, we have:')
print('Galactic longitude:', el, 'Galactic latitude:', be)

With right ascension = HERA latitude and declination = HERA longitude, we have:
Galactic longitude: 77.522971004297 Galactic latitude: -25.693447620731042


In [13]:
def interp(J_section, el, be):
    '''
    Abbreviation and conversion function. We get the interpolation value
    assuming @al and @be are in degrees.
    '''
    colatitude = np.radians(90 - el)
    longitude = np.radians(be)
    return hp.get_interp_val(J_section, colatitude, longitude)

In [14]:
def bad_interp(J_section, alt, az):
    return hp.get_interp_val(J_section, alt, az, lonlat=True)

In [16]:
lst = 5.9147
az, alt = rotations.eq_to_topo(np.degrees(lst), rotations.hera_lon, rotations.hera_lat, np.degrees(lst))
print('HERA altitude:', az)
print('HERA azimuth:', alt)

HERA altitude: 4.0279720258405205e-15
HERA azimuth: 37.85027777777778


In [17]:
# Hard-coded corrections
altitude = -56.0848
azimuth = 263.1775

In [21]:
rotations.ha_to_gal(rotations.get_lst(), rotations.hera_lon, rotations.get_lst(), radians=False)
#rotations.lon

(107.27056014103779, -39.87396161647384)